# Preparación para el TF MDS

## Preparación de los datos

Repo para trabajo de preparación

Dataset: https://www.yelp.com/dataset/documentation/main

**Requiere ejecución del notebook '00_EDA.ipynb'** para generar el archivo checkpoints/df5.pkl

Se crean dos repositorios de fotos recortadas al tamaño photo_size x photo_size:
- dataset/photos (todas las fotos)
- dataset/photos (solo 100 fotos por categoría)

In [2]:
import importlib
import image_preparation
importlib.reload(image_preparation)

/usr/lib/python3/dist-packages/requests/__init__.py:89: RequestsDependencyWarning: urllib3 (1.26.11) or chardet (3.0.4) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({}) doesn't match a supported "


<module 'image_preparation' from '/Projetos_dev/image_preparation.py'>

## Crea repo con fotos validas recortadas 224 x 224

In [3]:
image_preparation.main(dataset_photos_path = 'dataset/photos',
    dataset_output_path = 'dataset/recortadas',
    photo_size = 224,
    df_pickle_file='checkpoints/df5.pkl',
    max_photos=0)

Creando carpeta dataset/recortadas/food.
Creando carpeta dataset/recortadas/inside.
Creando carpeta dataset/recortadas/outside.
Creando carpeta dataset/recortadas/drink.
Creando carpeta dataset/recortadas/menu.


Photos cropped:   0%|          | 0/196660 [00:00<?, ?it/s]

## Crear toy dataset con 100 fotos por categoría

In [4]:
image_preparation.main(dataset_photos_path = 'dataset/photos',
    dataset_output_path = 'dataset/toy',
    photo_size = 224,
    df_pickle_file='checkpoints/df5.pkl',
    max_photos=100)

Carpeta dataset/toy/food existe. Vaciandola.
Carpeta dataset/toy/inside existe. Vaciandola.
Carpeta dataset/toy/outside existe. Vaciandola.
Carpeta dataset/toy/drink existe. Vaciandola.
Carpeta dataset/toy/menu existe. Vaciandola.


Photos cropped:   0%|          | 0/500 [00:00<?, ?it/s]